In [1]:
import os

from dotenv import load_dotenv
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings
from pymongo import MongoClient

load_dotenv()

/Users/sookhyun/Documents/personal/langchain-rag-poc/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [2]:
client = MongoClient(os.environ["DEV_MONGO_CONNECTION_STRING"])

In [3]:
# Control Group
control_vector_store = MongoDBAtlasVectorSearch(
    collection=client["lab_dev"]["langchain_existing_embedding"],
    index_name="test_control_vector_store_index",
    relevance_score_fn="euclidean",
    embedding=OpenAIEmbeddings(model="text-embedding-3-large", dimensions=2048),
)

In [6]:
# Treatment Group
treatment_vector_store = MongoDBAtlasVectorSearch(
    collection=client["lab_dev"]["langchain_embedding"],
    index_name="test_vector_store_index",
    relevance_score_fn="cosine",
    embedding=OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536),
)

# Also can be done manually through console
# vector_store.create_vector_search_index(dimensions=1536)

In [7]:
control_retriever = control_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})
treatment_retriever = treatment_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [14]:
def compare(query: str):
    print("Control Group")
    documents = control_retriever.invoke(query)
    print(",".join(list(map(lambda x: x.metadata.get("content_id"), documents))))
    
    print("Treatment Group")
    documents = treatment_retriever.invoke(query)
    print(",".join(list(map(lambda x: x.metadata.get("content_id"), documents))))

In [33]:
compare("서울에서 가기 좋은 미술 전시회")

Control Group
674d790fb402910f7bf63190,67554bbfc500f7140346ed12,674d78a1b402910f7bf6318d,674d7039b402910f7bf6308d,674d71953f07b6292e4a77d2,674d7a118809430f63298bc7,674d78133f07b6292e4a78bc,674d77358809430f63298b5d,67517031cafd7927d7d43e02,674d2e183f07b6292e4a6cce
Treatment Group
674d790fb402910f7bf63190,674d7a118809430f63298bc7,67554bbfc500f7140346ed12,674d78a1b402910f7bf6318d,674d71953f07b6292e4a77d2,674d77358809430f63298b5d,67506cc57afff00eaba410e4,674d7039b402910f7bf6308d,674d947d3f07b6292e4a7d3f,674d903db402910f7bf634d6


In [20]:
compare("서울 근교에 단풍보며 산책하기 좋은 공간")

Control Group
6752ae8e71c8454e08505e72,674db53d8809430f6329948e,6749ec708809430f63291e58,6750c46344b8a54a7e5e86a0,6752ac0fcafd7927d7d45190,6752a9cf592588034c7838cd,674ed755a4f99b3ec727fe28,674b154cb402910f7bf5e516,6755676271c8454e0850903b,6752aae1cafd7927d7d4516d
Treatment Group
674ea78a10f2a32e7a0c7e27,674ae83d8809430f63293a20,675031ce7afff00eaba40ce7,674e63f23f07b6292e4a958b,6750c61544b8a54a7e5e86b9,674d71953f07b6292e4a77d2,674d78a1b402910f7bf6318d,6753d3fd592588034c784bcf,674e885035d15703bb026f0e,675044a144b8a54a7e5e8078


In [21]:
compare("서울에서 눈올때 갈만한 공간")

Control Group
674db53d8809430f6329948e,6752ac0fcafd7927d7d45190,674caec1b402910f7bf61977,674db7803f07b6292e4a827f,6752aae1cafd7927d7d4516d,6752a9cf592588034c7838cd,674e63f23f07b6292e4a958b,6750c46344b8a54a7e5e86a0,6752ae8e71c8454e08505e72,6749ec708809430f63291e58
Treatment Group
67554bbfc500f7140346ed12,674d78a1b402910f7bf6318d,67506cc57afff00eaba410e4,674d7039b402910f7bf6308d,674eb24710f2a32e7a0c7fd2,674d2e183f07b6292e4a6cce,674d71953f07b6292e4a77d2,6752aae1cafd7927d7d4516d,674ea78a10f2a32e7a0c7e27,65d803ad46e44922f61d6207


In [22]:
compare("서울에의 한국적인 공간")

Control Group
674db53d8809430f6329948e,6752aae1cafd7927d7d4516d,674c0ce95383694484d06822,674d78a1b402910f7bf6318d,6749ec708809430f63291e58,6752a9cf592588034c7838cd,674d71953f07b6292e4a77d2,6750580cab01931e01bc2218,6750c46344b8a54a7e5e86a0,674ed76e76b183744d64808f
Treatment Group
674d78a1b402910f7bf6318d,674d947d3f07b6292e4a7d3f,674d71953f07b6292e4a77d2,67554bbfc500f7140346ed12,6752aae1cafd7927d7d4516d,67506cc57afff00eaba410e4,67501c297afff00eaba40b57,673754ff5d2e1522d8e0b708,674d7039b402910f7bf6308d,674eb24710f2a32e7a0c7fd2


In [23]:
compare("서울에서 친구와 단둘이 가기좋은 술집")

Control Group
675104b97afff00eaba416fb,6752df61cafd7927d7d454f0,674db53d8809430f6329948e,6753babdc500f7140346ce66,674ee8503f9bca1dab1bdf66,6753e3c3cafd7927d7d465d4,674d3e503f07b6292e4a7095,675057337afff00eaba40f37,6752537271c8454e0850570b,674c5e3b8809430f63296960
Treatment Group
6752df61cafd7927d7d454f0,65d803ad46e44922f61d6207,6749d9d18809430f63291bae,65f8084a2384e0486b5a0eaf,674d3e503f07b6292e4a7095,67563c9acafd7927d7d4912b,674ee8503f9bca1dab1bdf66,675104b97afff00eaba416fb,675044a144b8a54a7e5e8078,6752de17cafd7927d7d454ec


In [24]:
compare("서울의 실내 공간")

Control Group
6749ec708809430f63291e58,674c0ce95383694484d06822,674db53d8809430f6329948e,67554bbfc500f7140346ed12,674d71953f07b6292e4a77d2,6752aae1cafd7927d7d4516d,6750580cab01931e01bc2218,674ae83d8809430f63293a20,674db7803f07b6292e4a827f,674caec1b402910f7bf61977
Treatment Group
674d71953f07b6292e4a77d2,674d78a1b402910f7bf6318d,67554bbfc500f7140346ed12,674eb24710f2a32e7a0c7fd2,674d7039b402910f7bf6308d,674ae83d8809430f63293a20,674f0358f5815308ea301717,6752aae1cafd7927d7d4516d,67506cc57afff00eaba410e4,674db6f1b402910f7bf63ac2


In [25]:
compare("성탄절 술집")

Control Group
6753babdc500f7140346ce66,6752de17cafd7927d7d454ec,674b386cb402910f7bf5e995,674ea0f66841635da4c2f752,674ee8503f9bca1dab1bdf66,674ec1204cca144d176525d9,6752df61cafd7927d7d454f0,675196eb71c8454e08504dcb,6752a4c2c500f7140346bdc6,674bfcaa8809430f63295a90
Treatment Group
65f8084a2384e0486b5a0eaf,6752de17cafd7927d7d454ec,6755676271c8454e0850903b,65d8364646e44922f61d6257,65c0a7c6e1392d02a6da64c6,670786447ad7b5463ba51ede,674c5e3b8809430f63296960,6683c3c4d81428665e0bb347,674d3e503f07b6292e4a7095,674db6535383694484d09eef
